In [1]:
# import psycopg2
import pandas as pd
import numpy as np
import spacy
import stanfordnlp
import time
import scispacy
from tqdm import tqdm
import itertools
import pickle

import sys
import re, nltk
import os
from heuristic_tokenize import sent_tokenize_rules

In [90]:
#setting sentence boundaries
def sbd_component(doc):
    for i, token in enumerate(doc[:-2]):
        # define sentence start if period + titlecase token
        if token.text == '.' and doc[i+1].is_title:
            doc[i+1].sent_start = True
        if token.text == '-' and doc[i+1].text != '-':
            doc[i+1].sent_start = True
    return doc

#convert de-identification text into one token
def fix_deid_tokens(text, processed_text):
    deid_regex  = r"\[\*\*.{0,15}.*?\*\*\]" 
    if text:
        #print("!!!!!")
        indexes = [m.span() for m in re.finditer(deid_regex,text,flags=re.IGNORECASE)]
    else:
        #print("??????")
        indexes = []
    #print(indexes)
    for start,end in indexes:
        a = processed_text.merge(start_idx=start,end_idx=end)
        #print(a)
    #print(text)
    return processed_text
    

def process_section(section, note, processed_sections):
    # perform spacy processing on section
    #print("--------- before ---------")
    #print(section['sections'])
    processed_section = nlp(section['sections'])
    #print("--------- AFTER ----------")
    #print(processed_section)
    processed_section = fix_deid_tokens(section['sections'], processed_section)
    processed_sections.append(processed_section)

def process_note_helper(note):
    # split note into sections
    note_sections = sent_tokenize_rules(note)
    processed_sections = []
    section_frame = pd.DataFrame({'sections':note_sections})
    
    section_frame.apply(process_section, args=(note,processed_sections,), axis=1)
    return(processed_sections)

def process_text(sent, note):
    sent_text = sent['sents'].text
    sent_text = sent_text.replace("_","")
    sent_text = sent_text.replace("#","")
    sent_text = re.sub(r"(1[0-2]|[1-9]):[0-5][0-9] *(a|p|A|P)(m|M) *", "",sent_text)
    if len(sent_text) > 0 and sent_text.strip() != '\n':
        if '\n' in sent_text:
            sent_text = sent_text.replace('\n', ' ')
#         print('sent_text:', sent_text)
        if sent_text[-1] ==':':
            note['HEADERS'] += sent_text[:-1] + ','
#         else:
        note['TEXT'] += sent_text + '\n'
    
    
    

def get_sentences(processed_section, note):
    # get sentences from spacy processing
    sent_frame = pd.DataFrame({'sents': list(processed_section['sections'].sents)})
    sent_frame.apply(process_text, args=(note,), axis=1)


def process_note(note):
    try:
        note_text = note['TEXT'] #unicode(note['text'])
        note['TEXT'] = ''
        processed_sections = process_note_helper(note_text)
        ps = {'sections': processed_sections}
        ps = pd.DataFrame(ps)
        ps.apply(get_sentences, args=(note,), axis=1)
        return note 
    except Exception as e:
        print('!!!!! error', e)
        pass 

In [91]:
# import en_core_sci_md
# nlp = en_core_sci_md.load()
nlp = spacy.load('en_core_sci_md', disable=['tagger','ner'])
nlp.add_pipe(sbd_component, before='parser') 

In [92]:
SAMPLE_20K = pd.read_csv('SAMPLE_20K.csv')

In [93]:
SAMPLE_20K['TEXT'] = SAMPLE_20K['TEXT'].replace(to_replace=r"\[.*?\]", value="", regex=True)

In [97]:
# SAMPLE_20K = SAMPLE_20K.drop(columns=['Unnamed: 0'])

In [98]:
SAMPLE_20K['HEADERS'] = ""

In [99]:
print('Number of notes: %d' %len(SAMPLE_20K.index))
SAMPLE_20K['ind'] = list(range(len(SAMPLE_20K.index)))

Number of notes: 20000


In [100]:
tqdm.pandas()

In [101]:
SAMPLE_20K = SAMPLE_20K.progress_apply(process_note, axis=1)

  0%|          | 4/20000 [00:01<2:29:06,  2.24it/s]

!!!!! error 'NoneType' object has no attribute 'groups'


  0%|          | 10/20000 [00:05<4:10:24,  1.33it/s]

!!!!! error 'NoneType' object has no attribute 'groups'


  0%|          | 35/20000 [00:38<6:50:01,  1.23s/it]

...

100%|█████████▉| 19998/20000 [9:55:00<00:02,  1.36s/it]

!!!!! error 'NoneType' object has no attribute 'groups'


100%|██████████| 20000/20000 [9:55:01<00:00,  1.18s/it]

!!!!! error 'NoneType' object has no attribute 'groups'


100%|██████████| 20000/20000 [9:55:07<00:00,  1.79s/it]


In [102]:
keep = SAMPLE_20K.dropna()

In [104]:
discard_lst = {'','Admission Date', 'Discharge Date','Date of Birth', 'Phone','Date/Time', 'ID',
               'Completed by', 'Dictated By','Attending', 'Provider: ','Provider', 'Primary', 'Secondary', 
               ' MD Phone',' M.D. Phone', ' MD',' PHD',
               ' X', ' IV', ' VI', 'III', 'II', 'VIII',
               'JOB#','JOB#: cc', '# Code',
               'Metoprolol Tartrate 25 mg Tablet Sig',')','000 unit/mL Suspension Sig',' ','0.5 % Drops ','   Status: Inpatient DOB', 'Levothyroxine 50 mcg Tablet Sig', '0.5 % Drops Sig','Lidocaine 5 %(700 mg/patch) Adhesive Patch','Clopidogrel Bisulfate 75 mg Tablet Sig','Levofloxacin 500 mg Tablet Sig','Albuterol 90 mcg/Actuation Aerosol ','None Tech Quality: Adequate Tape #','000 unit/mL Solution Sig',
              }
keep['HEADERS'] = keep['HEADERS'].apply(lambda x: x.split(','))
keep['HEADERS'] = keep['HEADERS'].apply(lambda x: list(set(x).difference(discard_lst)))

In [106]:
# get top 10 sentences and get 25 vocabularies from those sentences
all_tokens = []
num_vocabs = 25
num_sentences = 10
discard_tokens = ('Sex: F', 'Sex: M', 'MEDICINE', 'Neurosurgery', 'Service: CT', 'Allergies')
for index, row in keep.iterrows():
    cur = [ ["<PAD>"] * num_vocabs for i in range(num_sentences) ]
    sents = row["TEXT"].splitlines()
    headers = ' '.join(row['HEADERS']).split()
        
    """
    for j in range(len(headers)):
        if j < num_vocabs:
            cur[0][j] = headers[j]       
    """          
    
    sentence_idx = 0
    for i in range(0, len(sents)):
        if sentence_idx >= num_sentences:
            break
        
        if sents[i].startswith(discard_tokens):
            continue
        
        vocabs = sents[i].split()
        
        for j in range(len(vocabs)):  
            if(j >= num_vocabs):
                break
            
            #print(str(sentence_idx)+" "+str(j)+" "+vocabs[j])
            cur[sentence_idx][j] = vocabs[j]
        
        sentence_idx += 1
    
    all_tokens.append(list(itertools.chain.from_iterable(cur)))
keep['padded_tokens'] = all_tokens

In [107]:
keep

,HADM_ID,TEXT,CATEGORY,DIAG_CODES,PROC_CODES,HEADERS,ind,padded_tokens
0,190979.0,Admission Note mom is very supportive.\nOther...,Nursing/other Nursing/other Nursing/other Nurs...,"V3000,V502",640.0,"[PE, Neuro, GU, Alt Parenting, Alt FEN]",0.0,"[Admission, Note, mom, is, very, supportive., ..."
1,187350.0,Admission Date:\nDischarge Date:\nDate of Birt...,Discharge summary Radiology Radiology Radiolog...,"5770,27651,00845,57450,4019,25000","5185.0,5187.0,3893.0","[Delayed Release(E.C.) Sig, Followup Instructi...",1.0,"[Admission, Date:, <PAD>, <PAD>, <PAD>, <PAD>,..."
3,133981.0,Admission Date:\nDischarge Date:\nDate of Birt...,Discharge summary ECG ECG Radiology Radiology ...,"1976,5845,34830,1977,78951,1572,56089,V4986,27...",5491.0,[Medications (reconciled with pt and records...,3.0,"[Admission, Date:, <PAD>, <PAD>, <PAD>, <PAD>,..."
4,147056.0,Admission Date:\nDischarge Date:\nDate of Birt...,Discharge summary Radiology Radiology Radiolog...,"5712,45620,30500,2875,135,5723,2720,49390","4233.0,9905.0,9904.0","[History of Present Illness, Past Medical Hist...",4.0,"[Admission, Date:, <PAD>, <PAD>, <PAD>, <PAD>,..."
5,148310.0,Neonatology Patient is 3.41 kg term newborn se...,Nursing/other Nursing/other,"V3001,V502,V290,V053","640.0,9955.0",[],5.0,"[Neonatology, Patient, is, 3.41, kg, term, new..."
...,...,...,...,...,...,...,...,...
19993,153176.0,BABYGRAM (CHEST ONLY) ...,Radiology Nursing/other Nursing/other Nursing/...,"V3001,7852,V053","640.0,9955.0",[PNS],19993.0,"[BABYGRAM, (CHEST, ONLY), Clip, <PAD>, <PAD>, ..."
19994,131551.0,Admission Date:\nDischarge Date:\nDate of Birt...,Discharge summary ECG ECG ECG ECG ECG ECG ECG ...,"53021,34830,51881,5070,4821,00845,78550,5990,V...","4516.0,966.0,3897.0,9604.0,9671.0,3891.0","[IMPRESSION: AP chest compared to , Primary D...",19994.0,"[Admission, Date:, <PAD>, <PAD>, <PAD>, <PAD>,..."
19995,152254.0,Admission Date:\nDischarge Date:\nService: MED...,Discharge summary Echo ECG ECG ECG Radiology R...,"0389,5070,78552,51881,5307,34982,70724,48241,5...","9671.0,9604.0,3891.0,3893.0,3897.0,14.0,966.0",[The following changes were made to your medic...,19995.0,"[Admission, Date:, <PAD>, <PAD>, <PAD>, <PAD>,..."
19997,116726.0,Admission Date:\nDischarge Date:\nDate of Birt...,Discharge summary Discharge summary Echo ECG E...,"3963,41401,39891,40391,5856,4260,42731,25040,V...","3524.0,3611.0,3772.0,3615.0,3783.0,3964.0,3995...","[INTERPRETATION, History of Present Illness, A...",19997.0,"[Admission, Date:, <PAD>, <PAD>, <PAD>, <PAD>,..."


In [108]:
keep.to_pickle("./Yixue.pkl")
keep.to_csv('Yixue.csv',index=True)

In [4]:
keep = pd.read_csv("Yixue.csv")

In [6]:
keep = keep.drop(columns=['Unnamed: 0'])

# Yezhou

In [63]:
keep2 = SAMPLE_20K.dropna()

In [7]:
def clean(text):
    output = re.sub(r"\n", " ",text)
    output = re.sub(r" +", " ",output)
    output = output.strip()
    return output

In [8]:
keep['CLEANED TEXT'] = keep['TEXT'].apply(clean)

In [9]:
import string
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
def remove_stopwords(text): 
        stop_words = set(stopwords.words("english")) 
        word_tokens = word_tokenize(text) 
        filtered_text = [word for word in word_tokens if word not in stop_words] 
        return filtered_text 
    
def preprocess(note):
    note = note.replace('\n',' ')
    note = note.replace('w/', 'with')
    note = note.lower() #lower case
    note = re.sub(r'\d+', '', note) #remove numbers
    note = note.translate(str.maketrans('', '', string.punctuation)) #remove punctuation
    note = " ".join(note.split())
    note = remove_stopwords(note)
    return note

In [10]:
discard_lst = {'','Admission Date', 'Discharge Date','Date of Birth', 'Phone','Date/Time', 'ID',
               'Completed by', 'Dictated By','Attending', 'Provider: ','Provider', 'Primary', 'Secondary', 
               ' MD Phone',' M.D. Phone', ' MD',' PHD',
               ' X', ' IV', ' VI', 'III', 'II', 'VIII',
               'JOB#','JOB#: cc', '# Code',
               'Metoprolol Tartrate 25 mg Tablet Sig',')','000 unit/mL Suspension Sig',' ','0.5 % Drops ','   Status: Inpatient DOB', 'Levothyroxine 50 mcg Tablet Sig', '0.5 % Drops Sig','Lidocaine 5 %(700 mg/patch) Adhesive Patch','Clopidogrel Bisulfate 75 mg Tablet Sig','Levofloxacin 500 mg Tablet Sig','Albuterol 90 mcg/Actuation Aerosol ','None Tech Quality: Adequate Tape #','000 unit/mL Solution Sig',
              }
# keep['HEADERS'] = keep['HEADERS'].apply(lambda x: x.split(','))
keep['HEADERS'] = keep['HEADERS'].apply(lambda x: list(set(x).difference(discard_lst)))

In [11]:
keep['CLEAN_WORDS'] = keep['TEXT'].apply(preprocess)

In [14]:
from nltk.stem import *
stemmer = PorterStemmer()


cat


In [19]:
keep['STEM_WORDS'] = keep['CLEAN_WORDS'].apply(lambda x: [stemmer.stem(w) for w in x])

In [21]:
corpus = {}
for i, s in enumerate(keep['STEM_WORDS']):
    for w in s:
        corpus[w] = corpus.get(w, 1) + 1
corpus = {k: v for k, v in sorted(corpus.items(), key=lambda item: item[1], reverse=True)}

In [45]:
len(corpus)

351134

In [46]:
corpus_slice = dict(itertools.islice(corpus.items(), 10000))

In [47]:
len(corpus_slice)

10000

In [48]:
# build dictionary
word2idx = {'<PAD>': 0, '<UNK>':1}
idx2word = {0: '<PAD>', 1:'<UNK>'}
for c in corpus_slice:
    word2idx[c] = len(word2idx)
    idx2word[len(idx2word)] = c

In [49]:
length = 256
def note2idx_cap(org_lst):
    coded_lst = []
    for w in org_lst:
        if len(coded_lst) < length and w in word2idx:
            coded_lst.append(word2idx[w])
        else:
            coded_lst.append(1)
    coded_lst += [0]*(length-len(coded_lst))
    return coded_lst

In [50]:
keep['CODED_TEXT'] = keep['STEM_WORDS'].apply(note2idx_cap) 

In [52]:
keep = keep.drop(columns=['HEADERS'])

In [53]:
keep.head()

,HADM_ID,TEXT,CATEGORY,DIAG_CODES,PROC_CODES,ind,padded_tokens,CLEANED TEXT,CLEAN_WORDS,STEM_WORDS,CODED_TEXT
0,190979.0,Admission Note mom is very supportive.\nOther...,Nursing/other Nursing/other Nursing/other Nurs...,"V3000,V502",640.0,0.0,"['Admission', 'Note', 'mom', 'is', 'very', 'su...",Admission Note mom is very supportive. Otherwi...,"[admission, note, mom, supportive, otherwise, ...","[admiss, note, mom, support, otherwis, noncont...","[65, 13, 790, 89, 680, 2640, 1, 2640, 1, 230, ..."
1,187350.0,Admission Date:\nDischarge Date:\nDate of Birt...,Discharge summary Radiology Radiology Radiolog...,"5770,27651,00845,57450,4019,25000","5185.0,5187.0,3893.0",1.0,"['Admission', 'Date:', '<PAD>', '<PAD>', '<PAD...",Admission Date: Discharge Date: Date of Birth:...,"[admission, date, discharge, date, date, birth...","[admiss, date, discharg, date, date, birth, se...","[65, 120, 77, 120, 120, 967, 1096, 316, 281, 1..."
2,133981.0,Admission Date:\nDischarge Date:\nDate of Birt...,Discharge summary ECG ECG Radiology Radiology ...,"1976,5845,34830,1977,78951,1572,56089,V4986,27...",5491.0,3.0,"['Admission', 'Date:', '<PAD>', '<PAD>', '<PAD...",Admission Date: Discharge Date: Date of Birth:...,"[admission, date, discharge, date, date, birth...","[admiss, date, discharg, date, date, birth, se...","[65, 120, 77, 120, 120, 967, 1096, 316, 1226, ..."
3,147056.0,Admission Date:\nDischarge Date:\nDate of Birt...,Discharge summary Radiology Radiology Radiolog...,"5712,45620,30500,2875,135,5723,2720,49390","4233.0,9905.0,9904.0",4.0,"['Admission', 'Date:', '<PAD>', '<PAD>', '<PAD...",Admission Date: Discharge Date: Date of Birth:...,"[admission, date, discharge, date, date, birth...","[admiss, date, discharg, date, date, birth, se...","[65, 120, 77, 120, 120, 967, 1096, 316, 1226, ..."
4,148310.0,Neonatology Patient is 3.41 kg term newborn se...,Nursing/other Nursing/other,"V3001,V502,V290,V053","640.0,9955.0",5.0,"['Neonatology', 'Patient', 'is', '3.41', 'kg',...",Neonatology Patient is 3.41 kg term newborn se...,"[neonatology, patient, kg, term, newborn, sent...","[neonatolog, patient, kg, term, newborn, sent,...","[1282, 6, 368, 1590, 2064, 312, 1413, 268, 438..."


In [54]:
import pickle
# keep.to_pickle("./May6.pkl")
keep.to_csv('May6.csv',index=True)

In [55]:
with open('word2idx.pickle','wb') as f:
    pickle.dump(word2idx,f,pickle.HIGHEST_PROTOCOL)
with open('idx2word.pickle','wb') as f:
    pickle.dump(idx2word,f,pickle.HIGHEST_PROTOCOL)

In [84]:
keep2.columns

Index(['HADM_ID', 'CATEGORY', 'TEXT', 'DIAG_CODES', 'PROC_CODES', 'CODED_DIAG',
       'CODED_PROC', 'HEADERS', 'ind', 'CLEANED TEXT', 'CLEAN_WORDS',
       'CODED_TEXT'],
      dtype='object')

In [50]:
clean(keep2['TEXT'][0])

'CHEST (PORTABLE AP) Clip Reason: Eval placement of dobhoff Admitting Diagnosis: CELLULITIS MEDICAL CONDITION: 67 year old woman with new dobhoff REASON FOR THIS EXAMINATION: Eval placement of dobhoff WET READ: ASpf SAT Dobhoff in stomach. Right IJ appropriate. No acute intrathoracic process. A FINAL REPORT PORTABLE CHEST OF COMPARISON: Study of . FINDINGS: Dobbhoff tube terminates in the distal stomach. Heart size remains normal, and lungs are clear.'

In [53]:
keep2

,HADM_ID,CATEGORY,TEXT,DIAG_CODES,PROC_CODES,CODED_DIAG,CODED_PROC,HEADERS,ind
0,197778.0,Radiology,CHEST (PORTABLE AP) ...,"57142,5856,5762,5722,5845,6824,40391,45621,790...","5059.0,93.0,5011.0,4523.0,5185.0,5187.0,966.0,...","[61, 256, 39, 42, 230, 49, 130, 152, 40, 92, 5...","[84, 54, 89, 55, 2, 28, 28, 0, 2, 17, 1, 3]",,0.0
1,154221.0,Nursing/other,NPN NIGHTS\nAlt resp r/t prematurity: Remains ...,"V3001,76502,769,7718,7707,7742,7766,7798,36221","5300.0,9604.0,9672.0,9983.0,1424.0,9915.0","[28, 86, 59]","[1, 5, 30, 12]","FEN,",1.0
2,177919.0,Nursing/other,CSRU PROGRESS NOTE\nS/O:\n CV: AFIB up to ...,"41401,4111,42731,4280,78820,4019,25000,2720,V4364","3615.0,3612.0,3961.0","[2, 57, 1, 3, 120, 0, 4, 10, 105]","[10, 39, 7]","S/O,",2.0
3,111516.0,Radiology,CT CHEST W/CONTRAST; CT ABDOMEN W/CONTRAST ...,"80115,3314,8730,E8809,4019,2720,3320","966.0,9671.0","[150, 0, 10, 142]","[2, 3]","IMPRESSION,",3.0
4,196570.0,Nursing/other,Neonatology Attending Note\nProcedure Note:\nP...,"V3101,76503,769,7742,7793,76524,77981,77082,77...","9671.0,9604.0,9390.0,966.0,3893.0,9915.0,9983....","[180, 86, 59, 140, 285, 11, 9]","[3, 1, 20, 2, 0, 12, 30, 8]","Procedure Note,Procedure,",4.0
...,...,...,...,...,...,...,...,...,...
19995,161829.0,Discharge summary,Admission Date:\nDischarge Date:\nDate of Birt...,"4241,4254,4280,20190,9971,42731,2449,53081","3521.0,3961.0","[81, 79, 3, 54, 1, 17, 13]","[49, 7]","Admission Date,Discharge Date,Date of Birth,Al...",19995.0
19996,152050.0,Nursing/other,Nursing Progress Note\n1- O/A: Infant received...,"V3001,769,7742,76515,76524,77081,36221,7470,V290","9672.0,9604.0,9390.0,9983.0,3893.0","[28, 86, 59, 114, 294, 11]","[5, 1, 20, 30, 0]",,19996.0
19997,103464.0,ECG,Sinus bradycardia.\nLeft atrial abnormality.\n...,"41401,4111,4280,25060,3572,4019,4168,2720,7812","3613.0,3723.0,3615.0,3961.0,8856.0","[2, 57, 3, 92, 53, 0, 55, 10]","[22, 11, 10, 7, 6]",,19997.0
19998,135902.0,Nursing/other,Neonatology -\nNNP Progress Note\nis active wi...,"V3000,76503,769,7707,7742,2880,76075,7470,V290","9604.0,9672.0,9915.0,966.0,3893.0,3891.0,9983.0","[23, 86, 59, 294, 11]","[1, 5, 12, 2, 0, 9, 30]",,19998.0
